In [1]:
import sqlite3
import random
from datetime import date, timedelta
import names

In [2]:
conn = sqlite3.connect('ecommerce_actual.db')

In [3]:
cursor = conn.cursor()

## Schema Design

In [4]:
# Customer Table
cursor.execute('''
DROP TABLE IF EXISTS Customers
''')

cursor.execute('''
CREATE TABLE Customers (
    CustomerID SERIAL PRIMARY KEY,
    FirstName VARCHAR(50),
    LastName VARCHAR(50),
    Email VARCHAR(100),
    PhoneNumber VARCHAR(15)
)
''')

# Address Table
cursor.execute('''
DROP TABLE IF EXISTS Addresses
''')

cursor.execute('''
CREATE TABLE Addresses (
    AddressID INTEGER PRIMARY KEY,
    CustomerID INTEGER,
    Street TEXT,
    City TEXT,
    State TEXT,
    ZipCode TEXT,
    Country TEXT,
    FOREIGN KEY (CustomerID) REFERENCES Customers(CustomerID)
    )
''')

# Product Table
cursor.execute('''
DROP TABLE IF EXISTS Products
''')

cursor.execute('''
CREATE TABLE Products (
    ProductID INTEGER PRIMARY KEY,
    ProductName TEXT,
    Category TEXT
)
''')

# Product Variants Table
cursor.execute('''
DROP TABLE IF EXISTS ProductVariants
''')
cursor.execute('''
CREATE TABLE ProductVariants (
    VariantID INTEGER PRIMARY KEY,
    ProductID INTEGER,
    VariantName TEXT,
    LaunchDate DATE,
    DiscontinuedDate DATE,
    FOREIGN KEY (ProductID) REFERENCES Products(ProductID)
)
''')

# Price Table
cursor.execute('''
DROP TABLE IF EXISTS Prices
''')

cursor.execute('''
CREATE TABLE Prices (
    PriceID INTEGER PRIMARY KEY,
    VariantID INTEGER,
    Price REAL,
    EffectiveFrom DATE,
    EffectiveTo DATE,
    FOREIGN KEY (VariantID) REFERENCES ProductVariants(VariantID)
)
''')

# Orders Table
cursor.execute('''
DROP TABLE IF EXISTS Orders
''')

cursor.execute('''
CREATE TABLE Orders (
    OrderID INTEGER PRIMARY KEY,
    CustomerID INTEGER,
    OrderDate DATE,
    ShippingAddressID INTEGER,
    FOREIGN KEY (CustomerID) REFERENCES Customers(CustomerID),
    FOREIGN KEY (ShippingAddressID) REFERENCES Addresses(AddressID)
)
''')

# OrderDetails Table:
cursor.execute('''
DROP TABLE IF EXISTS OrderDetails
''')

cursor.execute('''
CREATE TABLE OrderDetails (
    OrderDetailID INTEGER PRIMARY KEY,
    OrderID INTEGER,
    VariantID INTEGER,
    Quantity INTEGER,
    UnitPrice REAL,
    FOREIGN KEY (OrderID) REFERENCES Orders(OrderID),
    FOREIGN KEY (VariantID) REFERENCES ProductVariants(VariantID)
)
''')



## Data Generation:

In [5]:
customers = []
addresses = []

for i in range(20):
    first_name = names.get_first_name()
    last_name = names.get_last_name()
    email = f"{first_name.lower()}.{last_name.lower()}@example.com"
    phone_number = f"+91{random.randint(1000000000, 9999999999)}"
    customers.append((i+1, first_name, last_name, email, phone_number))
    
    for j in range(random.randint(1, 3)):
            street = f"{random.randint(100, 999)} Main St"
            city = "Kanpur"
            state = "Uttar Pradesh"
            zip_code = f"{random.randint(100000, 999999)}"
            country = "India"
            addresses.append((len(addresses) + 1, i + 1, street, city, state, zip_code, country))


In [6]:
products = []
variants = []

for i in range(20):
    product_name = f"Product{i + 1}"
    category = "Category" + str(random.randint(1, 3))
    products.append((i + 1, product_name, category))
    
    for j in range(random.randint(1, 3)):
        variant_name = f"{product_name} Variant{j + 1}"
        launch_date = date.today() - timedelta(days=random.randint(365, 1825))
        discontinued_date = launch_date + timedelta(days=random.randint(30, 120))
        variants.append((len(variants) + 1, i + 1, variant_name, launch_date, discontinued_date))


In [7]:
prices = []

for variant in variants:
    for j in range(random.randint(1, 3)):
        price = round(random.uniform(100, 1000), 2)
        effective_from = date.today() - timedelta(days=random.randint(0, 365))
        effective_to = effective_from + timedelta(days=random.randint(30, 365))
        prices.append((len(prices) + 1, variant[0], price, effective_from, effective_to))

In [8]:
orders = []
order_details = []

for i in range(1000):
    customer_id = random.choice(customers)[0]
    order_date = date.today() - timedelta(days=random.randint(0, 1825))
    shipping_address_id = random.choice([addr[0] for addr in addresses if addr[1] == customer_id])
    orders.append((i + 1, customer_id, order_date, shipping_address_id))
    
    for j in range(random.randint(1, 5)):
        variant_id = random.choice(variants)[0]
        quantity = random.randint(1, 5)
        unit_price = random.choice([price[2] for price in prices if price[1] == variant_id])
        order_details.append((len(order_details) + 1, i + 1, variant_id, quantity, unit_price))

## Data Insert:

In [9]:
cursor.executemany('INSERT INTO Customers VALUES (?, ?, ?, ?, ?)', customers)
cursor.executemany('INSERT INTO Addresses VALUES (?, ?, ?, ?, ?, ?, ?)', addresses)
cursor.executemany('INSERT INTO Products VALUES (?, ?, ?)', products)
cursor.executemany('INSERT INTO ProductVariants VALUES (?, ?, ?, ?, ?)', variants)
cursor.executemany('INSERT INTO Prices VALUES (?, ?, ?, ?, ?)', prices)
cursor.executemany('INSERT INTO Orders VALUES (?, ?, ?, ?)', orders)
cursor.executemany('INSERT INTO OrderDetails VALUES (?, ?, ?, ?, ?)', order_details)

conn.commit()
conn.close()

## Checking:

In [10]:
conn = sqlite3.connect('ecommerce_actual.db')
cursor = conn.cursor()

In [11]:
output = conn.execute("""SELECT * FROM Customers""")
for row in output:
    print(row)

(1, 'Lottie', 'Alonso', 'lottie.alonso@example.com', '+915315287583')
(2, 'Arlene', 'Alejandro', 'arlene.alejandro@example.com', '+913428399025')
(3, 'Stephanie', 'Wood', 'stephanie.wood@example.com', '+915197706853')
(4, 'Gene', 'Cline', 'gene.cline@example.com', '+916231551796')
(5, 'Keith', 'Broun', 'keith.broun@example.com', '+913783569047')
(6, 'Cora', 'Labranche', 'cora.labranche@example.com', '+918172328951')
(7, 'Sharon', 'Arnett', 'sharon.arnett@example.com', '+915485941387')
(8, 'Robert', 'Sherlock', 'robert.sherlock@example.com', '+919424895442')
(9, 'Joanne', 'Collins', 'joanne.collins@example.com', '+916928777287')
(10, 'Micheal', 'Rodriguez', 'micheal.rodriguez@example.com', '+914435563478')
(11, 'Felicia', 'Longino', 'felicia.longino@example.com', '+912312934833')
(12, 'Madeline', 'Vig', 'madeline.vig@example.com', '+911951379374')
(13, 'Clara', 'Martin', 'clara.martin@example.com', '+911465129596')
(14, 'Sandra', 'Zamora', 'sandra.zamora@example.com', '+911052200658')
(